In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import shutil

In [7]:
batch_size=512
projection_dim=128


total_results = []

for task in ['mortality_90days', 'readmission_15days'] :
# for task in ['readmission_15days'] :
    for seed in range(2026, 2029) : 
        for modality in ['tabular', 'lab', 'note'] :
            if modality == 'note' : 
                suffixes = ['_FINETUNE', '_FREEZE']
            else : 
                suffixes = ['']
            for suffix in suffixes :
                result_dir = Path(f'Results/Linear_{modality}/Seed_{seed}/{task}_proj_{projection_dim}_batch_{batch_size}{suffix}')
                exp_dir = result_dir
                output_dir = exp_dir / 'outputs'
                model_dir = exp_dir / 'models'
                model_paths = list(model_dir.glob('linear_epoch*.pth'))
                # {epoch : model_path}
                model_paths = {int(p.stem.split('_')[1].replace('epoch', '')) : p for p in model_paths}
                
                best_output_save_path = output_dir / 'best_epoch.pkl'
                best_score_save_path = output_dir / 'best_epoch_scores.pkl'
                best_score =0
                save = True 
                for epoch in range(1, 101) : 
                    epoch_output_path = output_dir / f'epoch{epoch:03d}_scores.pkl'
                    if not epoch_output_path.exists() :
                        continue
                    epoch_output = pickle.load(open(epoch_output_path, 'rb'))
                    
                    valid_output = epoch_output['valid']
                    valid_score = valid_output['auroc'] + valid_output['auprc']
                    if valid_score > best_score : 
                        best_score = valid_score
                        best_epoch = epoch
                        best_epoch_output = epoch_output
                        # print(f"Best Epoch : {best_epoch} with Valid Score : {best_score:.4f} | AUROC : {valid_output['auroc']:.4f}, AUPRC : {valid_output['auprc']:.4f} | TEST AUROC : {epoch_output['test']['auroc']:.4f}, AUPRC : {epoch_output['test']['auprc']:.4f}")
                if save : 
                    shutil.copy(output_dir / f'epoch{best_epoch:03d}.pkl', best_output_save_path)
                    shutil.copy(output_dir / f'epoch{best_epoch:03d}_scores.pkl', best_score_save_path)
                    model_path = model_paths[best_epoch]
                    best_model_save_path = model_dir / 'best_model.pth'
                    shutil.copy(model_path, best_model_save_path)
                    print(f"Copied best epoch output to {best_output_save_path} & {best_score_save_path}")
                total_results.append({
                    'seed' : seed,
                    'task' : task,
                    'modality' : modality+suffix,
                    'best_epoch' : best_epoch,
                    'valid_auroc' : best_epoch_output['valid']['auroc'],
                    'valid_auprc' : best_epoch_output['valid']['auprc'],
                    'test_auroc' : best_epoch_output['test']['auroc'],
                    'test_auprc' : best_epoch_output['test']['auprc'],
                })
total_results_df = pd.DataFrame(total_results)
total_results_df

Copied best epoch output to Results/Linear_tabular/Seed_2026/mortality_90days_proj_128_batch_512/outputs/best_epoch.pkl & Results/Linear_tabular/Seed_2026/mortality_90days_proj_128_batch_512/outputs/best_epoch_scores.pkl
Copied best epoch output to Results/Linear_lab/Seed_2026/mortality_90days_proj_128_batch_512/outputs/best_epoch.pkl & Results/Linear_lab/Seed_2026/mortality_90days_proj_128_batch_512/outputs/best_epoch_scores.pkl
Copied best epoch output to Results/Linear_note/Seed_2026/mortality_90days_proj_128_batch_512_FINETUNE/outputs/best_epoch.pkl & Results/Linear_note/Seed_2026/mortality_90days_proj_128_batch_512_FINETUNE/outputs/best_epoch_scores.pkl
Copied best epoch output to Results/Linear_note/Seed_2026/mortality_90days_proj_128_batch_512_FREEZE/outputs/best_epoch.pkl & Results/Linear_note/Seed_2026/mortality_90days_proj_128_batch_512_FREEZE/outputs/best_epoch_scores.pkl
Copied best epoch output to Results/Linear_tabular/Seed_2027/mortality_90days_proj_128_batch_512/outputs

,seed,task,modality,best_epoch,valid_auroc,valid_auprc,test_auroc,test_auprc
0,2026,mortality_90days,tabular,77,0.819079,0.421106,0.820207,0.420561
1,2026,mortality_90days,lab,31,0.754396,0.315571,0.749036,0.307039
2,2026,mortality_90days,note_FINETUNE,3,0.817043,0.484890,0.821119,0.475583
3,2026,mortality_90days,note_FREEZE,99,0.714525,0.265017,0.696474,0.268350
4,2027,mortality_90days,tabular,87,0.817039,0.413502,0.817422,0.420754
5,2027,mortality_90days,lab,40,0.745361,0.297783,0.748570,0.298868
6,2027,mortality_90days,note_FINETUNE,3,0.820925,0.487611,0.822796,0.481710
7,2027,mortality_90days,note_FREEZE,99,0.687887,0.263925,0.711240,0.269011
8,2028,mortality_90days,tabular,60,0.828941,0.437157,0.815528,0.420176
9,2028,mortality_90days,lab,32,0.749146,0.309385,0.747210,0.301525
